Processamento da Consulta

A função query_process transforma a entrada inicial de uma pergunta em uma consulta. Palavras que não sejam nomes próprios (NPROP), números (NUM), verbos (V), substantivos (N) e adjetivos (ADJ) são removidos. Para a função é utilizado a biblioteca nlpnet.


In [ ]:
import nlpnet

def query_process(texto_entrada):
    #Diretorio dos modelos de etiquetação
    data_dir = 'pos-pt'
    #Definição do diretorio e linguagem a utilizar
    tagger = nlpnet.POSTagger(data_dir, language='pt')
    tagged_str = tagger.tag(texto_entrada)
    #print(tagged_str)

    texto_consulta = []
    for i in tagged_str:
        #print(i)
        for k in i:
            # print(k)
            tags = ['NPROP', 'NUM', 'V', 'ADJ', 'N']
            entrada = k
            for i in tags:
                # print(i)
                if entrada[1] == i:
                    texto_consulta.append(entrada[0])

    return str(texto_consulta)

Recuperação de informação

A função similaridade_sentencas recebe como entrada uma sentença(neste exemplo uma pergunta) é retorna uma passagem com maior valor de similaridade. Para esta função são utilizados TF-IDF e similaridade do cosseno.

Atenção: Antes de utilizar esta função carregar base de documento para se buscar a resposta similia

A função pre_processamento será mostrada em seguida.


In [ ]:
#---------------Atenção-------------------------------
corpus_texto = open('/content/estatuto_uespi.txt','r',encoding='utf8')
corpus = []
for i in corpus_texto:
    corpus.append(i)
corpus_join = ''.join(corpus)
corpus = corpus_join.split() 

lista_sentencas = nltk.sent_tokenize(corpus_replace)
len(lista_sentencas)
lista_sentencas_pre_processadas = []

for frases in lista_sentencas:
    lista_sentencas_pre_processadas.append(pre_processamento_texto(frases))

#-----------------------------------------------------


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# calcular similariedade do cosseno + TF-IDF
def similaridade_sentencas(sentenca):
    lista_similaridade = []
    # recebe a sentença para se analisar é ja faz o pré-processamento
    sentenca_entrada = pre_processamento_texto(sentenca)
    # adicionar a sentenca de entrada no conjunto do regimento
    lista_sentencas.append(sentenca)
    lista_sentencas_pre_processadas.append(sentenca_entrada)

    # convertendo base de dados para tf-idf
    tfidf = TfidfVectorizer()
    palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_pre_processadas)

    # calculo da similariedade
    # pega a ultima frase que foi adicionada que no caso e nossa 'sentenca_entrada'
    similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

    # ordena por ordem decrescente cada indice
    similaridade.argsort()

    # 5 maiores resultados de similariedade
    n1 = similaridade.argsort()[0][-2]

    lista_similaridade.append(str(lista_sentencas[n1]))
   
    # remove a sentenca adicionada
    del (lista_sentencas[-1])
    del (lista_sentencas_pre_processadas[-1])

    return str(lista_similaridade)

Função de pre_processamento.

Função de pre_processamento de texto, remove pontuações, stopwords, numeros.

In [8]:
#implementação das funcões para pré-processamento dos texto
import importlib
import re
import nltk
import string
import unicodedata
import sys
importlib.reload(sys)

#------------------------------------------------
stopwords_nltk = nltk.corpus.stopwords.words('portuguese')
#--------------------------------------------------
#-------------------------------------------------
#removendo pontuações
#removendo stopwords com nltk
#removendo numeros
#-----------------------------------------------
def pre_processamento_texto(texto):
    #texto = str(texto)
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    lista_de_palavras = []
    palavras = texto.split()
    for token in palavras:
        novo_token = regex.sub(u'',token)
        if not novo_token == u'':
            lista_de_palavras.append(novo_token)
    content = [w for w in lista_de_palavras if w.lower().strip() not in stopwords_nltk]

    texto_limpo = []
    for word in content:
        nfkd = unicodedata.normalize('NFKD', word)
        palavras_sem_acento = u''.join(c for c in nfkd if not unicodedata.combining(c))

        q = re.sub('[^a-zA-Z0-9 \\\]', ' ', palavras_sem_acento)

        texto_limpo.append(q.lower().strip())

    tokens = [t for t in texto_limpo if len(t)>2 and not t.isdigit()]
    result = ' '.join(tokens)

    return result